# Pipeline experiments

### Setup

In [130]:
# libraries
import pandas as pd
import numpy as np
import torch
import json
import os

from getpass import getpass
from langchain import HuggingFaceHub

from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import SequentialChain

import random

random.seed(42)

In [2]:
# logging in
HUGGINGFACEHUB_API_TOKEN = getpass()

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN


 ········


In [131]:
tokens = 64

llm_t5 = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"max_new_tokens": tokens, "temperature":0.1})#, "max_length": length})
llm_falcon_inst = HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"max_new_tokens": tokens, "temperature":0.1})#, "max_length": length})

llm = llm_t5

## Data generation

In [43]:
size = 600
dataset = pd.DataFrame(columns=['Sky','Temperature','Precipitation','Wind','Animal','Place'])
dataset['Sky'] = (np.random.choice(['clear', 'cloudy', 'overcast'], size, replace=True))
dataset['Temperature'] = (np.random.choice(range(-10, 30), size, replace=True))
precip = ([0]*(int(size/3)) + list(np.random.choice(range(0, 20), int(size/3*2), replace=True))) # 1/3 without rain (almost all rain seemed too much)
random.shuffle(precip)
dataset['Precipitation'] = precip
dataset['Wind'] = (np.random.choice(range(0, 80), size, replace=True))
dataset['Animal'] = (np.random.choice(['cat', 'bird','fish'], size, replace=True))
dataset['Place'] = (np.random.choice(['beach', 'city', 'meadows', 'mountains'], size, replace=True))
dataset

,Sky,Temperature,Precipitation,Wind,Animal,Place
0,overcast,6,4,23,bird,city
1,clear,-6,5,0,fish,mountains
2,overcast,28,12,54,cat,mountains
3,overcast,17,3,58,cat,city
4,cloudy,8,12,0,bird,city
...,...,...,...,...,...,...
595,clear,29,0,69,bird,meadows
596,cloudy,29,11,39,bird,mountains
597,cloudy,12,6,12,fish,mountains
598,clear,1,0,31,bird,meadows


In [98]:
#dataset.to_csv('weather_generated_600.csv')
dataset = pd.read_csv('weather_generated_600.csv')

## Content selection

In [6]:
# simple abduction - we do not tell explicitly for rain
CSP_simple_template = """The reasoning task is to select the more probable explanation, given the rules. 
There are three variables: [VAR:temp] [VAR:precip] and [VAR:sky].
Rules: If [VAR:temp] is less than zero, there is snow. If [VAR:precip] is zero then there is no rain and no snow. If [VAR:sky] is clear, there is no rain and no snow.
The three explanations are: There is no rain and no snow. There is snow. There is rain.
[VAR:temp] is {T}. [VAR:precip] is {P}. [VAR:sky] is {S}.
Answer:
"""
CSP_simple_selection = PromptTemplate.from_template(CSP_simple_template)
CSP_simple_chain = LLMChain(llm=llm, prompt=CSP_simple_selection, output_key="precip_simple")

In [7]:
# more complex 
CSP_template = """The reasoning task is to select the more probable explanation, given the rules. 
There are four variables: [VAR:temp] [VAR:precip], [VAR:sky] and [VAR:place].
Rules: If [VAR:temp] is less than zero, there is snow. If [VAR:precip] is zero then there is no rain and no snow. If [VAR:sky] is clear, there is no rain and no snow. 
If there is rain and the [VAR:place] is mountains, there is storm.
The four explanations are: There is no rain and no snow. There is snow. There is rain. There is storm.
[VAR:temp] is {T}. [VAR:precip] is {P}. [VAR:sky] is {S}. [VAR:place] is {L}.
Answer:
"""
CSP_selection = PromptTemplate.from_template(CSP_template)
CSP_chain = LLMChain(llm=llm, prompt=CSP_selection, output_key="precipitation")

In [8]:
# can use world knowledge
CSP_know_template = """The reasoning task is to select the more probable explanation, given the rules. 
There are two variables: temperature and temperature.
Rules: If temperature is less than zero, there is snow. If precipitation is zero then there is no rain and no snow. If sky is clear, there is no rain and no snow.
The three explanations are: There is no rain and no snow. There is snow. There is rain.
Temperature is {T}. Precipitation is {P}. The sky is {S}.
Answer: 
"""
CSP_know_selection = PromptTemplate.from_template(CSP_know_template)
CSP_know_chain = LLMChain(llm=llm, prompt=CSP_know_selection, output_key="precip_know")

In [11]:
# Generation template
CSP_gen_template = """The reasoning task is to decide what will happen, given the rules. 
Rules: If temperature is less than zero, there is snow. If precipitation is zero then there is no rain and no snow. If sky is clear, there is no rain and no snow.
Temperature is {T}. Precipitation is {P}. The sky is {S}.
Answer: 
"""
CSP_gen_selection = PromptTemplate.from_template(CSP_gen_template)
CSP_gen_chain = LLMChain(llm=llm, prompt=CSP_gen_selection, output_key="precip_gen")

In [18]:
# "nonsense" answer
CSP_pizza_template = """The reasoning task is to select the more probable explanation, given the rules. 
There are two variables: [VAR:temp] and [VAR:precip].
Rules: If [VAR:temp] is less than zero, there is snow. If [VAR:precip] is zero then there is no rain and no snow.
The four explanations are: There is no rain and no snow. There is pizza. There is snow. There is rain. 
[VAR:temp] is {T}. [VAR:precip] is {P}.
Answer:
"""
CSP_pizza_selection = PromptTemplate.from_template(CSP_pizza_template)
CSP_pizza_chain = LLMChain(llm=llm, prompt=CSP_pizza_selection, output_key="precip_pizza")


CSP_drizzle_template = """The reasoning task is to select the more probable explanation, given the rules. 
There are two variables: [VAR:temp] and [VAR:precip].
Rules: If [VAR:temp] is less than zero, there is snow. If [VAR:precip] is zero then there is no rain and no snow.
The four explanations are: There is no rain and no snow. There is drizzle. There is snow. There is rain. 
[VAR:temp] is {T}. [VAR:precip] is {P}.
Answer:
"""
CSP_drizzle_selection = PromptTemplate.from_template(CSP_drizzle_template)
CSP_drizzle_chain = LLMChain(llm=llm, prompt=CSP_drizzle_selection, output_key="precip_drizzle")


CSP_shower_template = """The reasoning task is to select the more probable explanation, given the rules. 
There are two variables: [VAR:temp] and [VAR:precip].
Rules: If [VAR:temp] is less than zero, there is snow. If [VAR:precip] is zero then there is no rain and no snow.
The four explanations are: There is no rain and no snow. There is shower. There is snow. There is rain. 
[VAR:temp] is {T}. [VAR:precip] is {P}.
Answer:
"""
CSP_shower_selection = PromptTemplate.from_template(CSP_shower_template)
CSP_shower_chain = LLMChain(llm=llm, prompt=CSP_shower_selection, output_key="precip_shower")


In [52]:
# we have when there is wind - needs calculator - higher/lower is by voko, 48,50 > 50, 3,4 > 5...
selection_CSW_template = """The reasoning task is to select the more probable explanation, given the rules. There is a variable: [VAR:wind].
Rules: If [VAR:wind] is less than 18 there is no wind. If [VAR:wind] is more than 50 there is strong wind.
The three explanations are: There is wind. There is no wind. There is strong wind.
[VAR:wind] is {W}.
Answer: 
"""
prompt_CSW_selection = PromptTemplate.from_template(selection_CSW_template)
CSW_selection_chain = LLMChain(llm=llm, prompt=prompt_CSW_selection, output_key="wind_select")

In [24]:
# Sky state
CSS_template = """The reasoning task is to select the more probable explanation, given the rules. There are three variables: [VAR:sky], [VAR:place] and [VAR:animal].
Rules: If [VAR:place] is beach, there is mist. If [VAR:place] is city, there is smog.
The three explanations are: There is mist. The sky is {S}. There is smog.
Values: [VAR:sky] is {S}. [VAR:place] is {L}. [VAR:animal] is {A}.
Answer: 
""" 
CSS_prompt = PromptTemplate.from_template(CSS_template)
CSS_chain = LLMChain(llm=llm, prompt=CSS_prompt, output_key="sky_select")

CSS_all_template = """The reasoning task is to select the more probable explanation, given the rules. There are three variables: [VAR:sky], [VAR:place] and [VAR:animal].
Rules: If [VAR:place] is beach and [VAR:animal] is cat, there is mist. If [VAR:place] is city or [VAR:animal] is bird, there is smog.
The three explanations are: There is mist. The sky is {S}. There is smog.
Values: [VAR:sky] is {S}. [VAR:place] is {L}. [VAR:animal] is {A}.
Answer: 
""" #
CSS_all_prompt = PromptTemplate.from_template(CSS_all_template)
CSS_all_chain = LLMChain(llm=llm, prompt=CSS_all_prompt, output_key="sky_all_select")

CSS_and_template = """The reasoning task is to select the more probable explanation, given the rules. There are three variables: [VAR:sky], [VAR:place] and [VAR:animal].
Rules: If [VAR:place] is beach and [VAR:animal] is cat, there is mist. If [VAR:place] is city, there is smog.
The three explanations are: There is mist. The sky is {S}. There is smog.
Values: [VAR:sky] is {S}. [VAR:place] is {L}. [VAR:animal] is {A}.
Answer: 
"""
CSS_and_prompt = PromptTemplate.from_template(CSS_and_template)
CSS_and_chain = LLMChain(llm=llm, prompt=CSS_and_prompt, output_key="sky_and_select")

CSS_or_template = """The reasoning task is to select the more probable explanation, given the rules. There are three variables: [VAR:sky], [VAR:place] and [VAR:animal].
Rules: If [VAR:place] is beach, there is mist. If [VAR:place] is city or [VAR:animal] is bird, there is smog.
The three explanations are: There is mist. The sky is {S}. There is smog.
Values: [VAR:sky] is {S}. [VAR:place] is {L}. [VAR:animal] is {A}.
Answer: 
""" 
CSS_or_prompt = PromptTemplate.from_template(CSS_or_template)
CSS_or_chain = LLMChain(llm=llm, prompt=CSS_or_prompt, output_key="sky_or_select")


In [21]:
# This works really badly !!!!! Removes too much stuff
selection_template = """Task: Remove sentences from text based on the provided rules.
Rules: If there is "no wind" in a sentence, remove it. If there is "no rain" in a sentence, remove it.
Text: {sky_select}. {precipitation2} {P} mm/h. The temperature is {T} °C. {wind_select} {W} km/h.
Answer:
"""
selection_prompt = PromptTemplate.from_template(selection_template)
selection_chain = LLMChain(llm=llm_t5, prompt=selection_prompt, output_key="selection")
# The task is to remove the sentences containing words "no wind" or "no rain and no snow" from given text. Output all other sentences.

# works
CSA_template = """Task: Remove sentences from text based on the provided rules. There is variable [VAR:animal].
Rules: If [VAR:animal] is fish then remove the sentence containing word "wind". If the [VAR:animal] is not fish, output all the provided text.
[VAR:animal] is {A}. 
Text: {ordering2}
Answer
"""
CSA_prompt = PromptTemplate.from_template(CSA_template)
CSA_chain = LLMChain(llm=llm_t5, prompt=CSA_prompt, output_key="selection_A")


## Discourse plan

In [92]:
ordering_template = """The reasoning task is to select the best order of event, given the rules. There are two variables: [VAR:animal] and [VAR:sky].
Rules: If [VAR:animal] is cat then the order is: T, S, P, W. If [VAR:sky] is clear then the order is: P, S, T, W.
If [VAR:animal] is bird then the order is: W, S, P, T. Usually the order is: S, P, T, W.
The possible explanations are: S, P, T, W; W, S, P, T; P, S, T, W; W, P, T, S; T, S, P, W;
[VAR:animal] is {A}. [VAR:sky] is {S}.
Answer:
"""
# if overcast and (bird or cat) there are more valid possibilities.
ordering_selection = PromptTemplate.from_template(ordering_template)
ordering_chain = LLMChain(llm=llm, prompt=ordering_selection, output_key="ordering")


# this part is a bit problematic?
ordering_template2 = """Task: Replace letters in a given sequence with their full corresponding sentences.
I will give you an example:
Context: W is sentence: "There is strong wind. 60 km/h." S is sentence: "The sky is cloudy." P is sentence: "There is no rain and no snow. 0 mm/h." T is sentence: "The temperature is 20 °C."
Sequence: T, S, P, W. Generate the sequence with letters replaced by their corresponding sentences and include all the provided sentences.
Answer: The temperature is 20 °C. The sky is cloudy. There is no rain and no snow 0 mm/h. There is strong wind 60 km/h.
Replace letters in a given sequence with their full corresponding sentences.
Context: W is sentence: "{wind_select} {W} km/h." S is sentence: "{sky_select}." P is sentence: "{precipitation2} {P} mm/h." T is sentence: "The temperature is {T} °C."
Sequence: {ordering}. Generate the sequence with the letters replaced by their corresponding sentences and include all the provided sentences.
Answer:
"""

# jsut old examples from trying few-shot
"""Context: Replace letter W with the sentence: "There is no wind. 2 km/h." Replace letter S with the sentence: "The sky is overcast." 
Replace letter P with the sentence: "There is snow. 5 mm/h." Replace letter T with the sentence: "The temperature is -3 °C."
Sequence: S, P, T, W. Generate the sequence with the letters replaced by their corresponding sentences and include all the provided sentences
Answer: The sky is overcast. There is snow 5 mm/h. The temperature is -3 °C. There is no wind 2 km/h.
Context: Replace letter W with the sentence: "There is wind. 24 km/h." Replace letter S with the sentence: "There is mist." 
Replace letter P with the sentence: "There is rain. 10 mm/h." Replace letter T with the sentence: "The temperature is 8 °C."
Sequence: P, S, T, W. Generate the sequence with the letters replaced by their corresponding sentences and include all the provided sentences
Answer: There is rain 10 mm/h. There is mist. The temperature is 8 °C. There is wind 24 km/h."""
ordering_selection2 = PromptTemplate.from_template(ordering_template2)
ordering_chain2 = LLMChain(llm=llm, prompt=ordering_selection2, output_key="ordering2")

## Generation

In [123]:
# well this quite works, T5 is not very wordy :) maybe can be combined... 
#realization_template = """Generate a fluent sentence in future tense. The sentence is about: rain {T} mm/h."""
#prompt_realization = PromptTemplate.from_template(realization_template)
# llm_falcon_inst #llm_t5
#LLMChain(llm=llm_t5, prompt=prompt_realization, output_key="selection").run({"T":3})

# The temperature will be 15 °C.  The wind will blow at 25 km/h
# not working with T5, Falcon works but hallucinate a bit (gentle breeze... moderate breeze...)
realization_template = """Task: Generate a fluent weather forecast in future tense from provided informations. Use all the informations but and the informations provided.
Keep the given order of information. Weather forecast must contain information about where it is.
Informations: Tomorrow in/on the {L}. {ordering2}. 
Answer: """
prompt_realization = PromptTemplate.from_template(realization_template)
realization_chain = LLMChain(llm=llm_t5, prompt=prompt_realization, output_key="realization")

realization_template2 = """Task: Generate a fluent weather forecast in future tense from provided informations. Use all the informations but and the informations provided.
Keep the given order of information. Weather forecast must contain information about where it is.
Informations: Tomorrow in/on the {L}. {ordering2}. 
Answer: """
# "Dear {A}" on the start makes a letter from it :D
# The forecast is for a place in the {L}. - cannot make it to say for where the forecast is.
prompt_realization2 = PromptTemplate.from_template(realization_template2)
realization_chain2 = LLMChain(llm=llm_falcon_inst, prompt=prompt_realization2, output_key="realization2")

## Run - with intermediate results

In [127]:
CSW, CSS, CSS_and, CSS_or, CSS_both, S1, S2, CSA, T5, Falcon = [], [], [], [], [], [],[],[], [],[]
CSP, CSP_simple, CSP_know, CSP_gen, CSP_pizza, CSP_drizzle, CSP_shower = [], [], [], [], [], [], []
for i, row in dataset.iterrows():
    falcon = realization_chain2.run({"L":row.Place, "ordering2":row.S2 })
    Falcon.append(falcon)
    """  # this is just to have the intermediate results
    t5 = realization_chain2.run({"L":row.Place, "ordering2":row.S2 })
    T5.append(t5)
    s1 = ordering_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    S1.append(s1)
    csp = CSP_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSP.append(csp)
    csp_simp = CSP_simple_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSP_simple.append(csp_simp)
    csp_k = CSP_know_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSP_know.append(csp_k)
    csp_g = CSP_gen_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSP_gen.append(csp_g)
    csp_piz = CSP_pizza_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSP_pizza.append(csp_piz)
    csp_dr = CSP_drizzle_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSP_drizzle.append(csp_dr)
    csp_sh = CSP_shower_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSP_shower.append(csp_sh)
    w = CSW_selection_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSW.append(w)
    css = CSS_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSS.append(css)
    css_and = CSS_all_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSS_and.append(css_and)
    css_or = CSS_or_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSS_or.append(css_or)
    css_both = CSS_all_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
    CSS_both.append(css_both)
    """

dataset['Falcon'] = Falcon
"""
dataset['T5'] = T5
dataset['CSA'] = CSA
dataset['S2'] = S2
dataset['CSP'] = CSP
dataset['CSP_simple'] = CSP_simple
dataset['CSP_know'] = CSP_know
dataset['CSP_gen'] = CSP_gen
dataset['CSP_pizza'] = CSP_pizza
dataset['CSP_drizzle'] = CSP_drizzle
dataset['CSP_shower'] = CSP_shower
dataset['CSW'] = CSW
dataset['CSS'] = CSS
dataset['CSS_and'] = CSS_and
dataset['CSS_or'] = CSS_or
dataset['CSS_both'] = CSS_both
dataset['S1'] = S1
"""
    
#S1_selection_chain

"\ndataset['T5'] = T5\ndataset['CSA'] = CSA\ndataset['S2'] = S2\ndataset['CSP'] = CSP\ndataset['CSP_simple'] = CSP_simple\ndataset['CSP_know'] = CSP_know\ndataset['CSP_gen'] = CSP_gen\ndataset['CSP_pizza'] = CSP_pizza\ndataset['CSP_drizzle'] = CSP_drizzle\ndataset['CSP_shower'] = CSP_shower\ndataset['CSW'] = CSW\ndataset['CSS'] = CSS\ndataset['CSS_and'] = CSS_and\ndataset['CSS_or'] = CSS_or\ndataset['CSS_both'] = CSS_both\ndataset['S1'] = S1\n"

In [128]:
dataset.head()

,ID,Sky,Temperature,Precipitation,Wind,Animal,Place,CSW,CSS,CSS_and,...,CSP_know,CSP_gen,CSP_pizza,CSP_drizzle,CSP_shower,S1,S2,CSA,T5,Falcon
0,0,overcast,6,4,23,bird,city,There is wind,There is smog,There is smog,...,There is rain.,There will be rain.,There is rain.,There is rain.,There is rain.,"W, S, P, T",There is wind 23 km/h. There is smog. There is...,There is smog. There is rain 4 mm/h. The tempe...,"Tomorrow in the city there is wind 23 km/h, sm...","\nTomorrow in the city, there will be wind at ..."
1,1,clear,-6,5,0,fish,mountains,There is no wind,The sky is clear,The sky is clear,...,There is snow,There is no snow.,There is snow,There is snow,There is snow,"P, S, T, W",There is snow 5 mm/h. The temperature is -6 °C...,There is snow 5 mm/h. The temperature is -6 °C...,Tomorrow in the mountains there is snow 5 mm/h...,"\nTomorrow on the mountains, there will be sno..."
2,2,overcast,28,12,54,cat,mountains,There is strong wind,The sky is overcast,There is mist,...,There is rain.,There will be snow.,There is rain.,There is rain.,There is rain.,"T, S, P, W",The temperature is 28 °C. The sky is overcast....,The temperature is 28 °C. The sky is overcast....,The temperature will be 28 °C. The sky will be...,"\nTomorrow, on the mountains, the temperature ..."
3,3,overcast,17,3,58,cat,city,There is strong wind,There is smog,There is smog,...,There is rain.,There is rain.,There is rain.,There is rain.,There is rain.,"T, S, P, W",The temperature is 17 °C. There is smog. There...,The temperature is 17 °C. There is smog. There...,"Tomorrow in the city there is smog, rain 3 mm/...",\nThe weather forecast for tomorrow in the cit...
4,4,cloudy,8,12,0,bird,city,There is no wind,There is smog,There is smog,...,There is rain.,There will be rain and snow.,There is rain.,There is rain.,There is rain.,"W, S, P, T",There is no wind 0 km/h. There is smog. There ...,There is no wind 0 km/h. There is smog. There ...,Tomorrow in the city there is no wind 0 km/h. ...,"\nTomorrow in the city, there will be no wind ..."


In [129]:
#dataset.to_csv('weather_results.csv')

## Run - with full pipeline

In [18]:
llm =llm_t5 # llm_falcon_inst #llm_t5

In [32]:
overall_chain = SequentialChain(chains=[S_selection_chain, P_chain2, W_selection_chain, ordering_chain, ordering_chain2],# selection_A_chain]
                                input_variables=["S","P", "T", "W","L","A"], verbose=True)

In [33]:
for i, row in dataset.sample(3).iterrows():
    sentences = overall_chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})

    t5 = realization_chain.run({"ordering2":sentences, "L":row.Place})
    falcon = realization_chain2.run({"ordering2":sentences, "L":row.Place})
    
    print(row.Temperature, row.Precipitation, row.Wind, row.Sky, row.Place, row.Animal)
    print(t5)
    print(falcon)



> Entering new  chain...

> Finished chain.
14 4 8 cloudy mountains fish
Tomorrow in the mountains the sky is cloudy, there is storm 4 mm/h, the temperature is 14 °C, there is no wind 8 km/h.

Tomorrow in the mountains, the sky will be cloudy with a chance of storming. The temperature will be around 14°C, and there will be no wind at 8 km/h.


> Entering new  chain...

> Finished chain.
-4 12 18 clear beach bird
Tomorrow in/on the beach. W, no rain and no snow 12 mm/h, The temperature is -4 °C, There is wind 18 km/h.

Tomorrow on the beach, the weather will be sunny with a comfortable temperature of -4 °C. There will be a gentle breeze blowing at 18 km/h, providing a pleasant day for beach-goers.


> Entering new  chain...

> Finished chain.
0 1 44 overcast beach cat
Tomorrow there is mist, there is snow 1 mm/h, the temperature is 0 °C, the wind is 44 km/h.

Tomorrow on the beach, there will be mist and snow with a thickness of 1 mm. The temperature will be around 0 °C, and there wil

# Legacy

In [ ]:
# ordering prompts
# I should somehow make the precip(rain,snow) together....
# add if hurricane it is first or something
ordering_template = """The task is to generate a correct sequence of variables, given the context.
The context is: Variable hurricane it is always first in the sequence. Variable temperature is usually first in the sequence. 
The variable wind is always last in the sequence.
The variables are {variables}. 
Answer:
"""
ordering_selection = PromptTemplate.from_template(ordering_template)
ordering_chain = LLMChain(llm=llm, prompt=ordering_selection, output_key="ordering")

# really hard to generate it!!!!
ordering_template = """"Order given words in a correct order based on provided rules. Provide reasoning.
Rules: Temperature is always first. Wind is always last.
I will give you an example:
Given words: wind, temperature, rain
Answer: The variable temperature should be first. Placing it first in the sequence: temperature, then wind, then rain. 
Wind should be last. Placing it last in the sequence: temperature, then rain, then wind. This is correct.
Given words: wind, snow.
Answer: The wind should be last. Placing it last: snow, then wind. This is correct.
Given words: {variables}
Answer: 
"""
ordering_selection = PromptTemplate.from_template(ordering_template)
ordering_chain = LLMChain(llm=llm, prompt=ordering_selection, output_key="ordering")


# ordering prompts
ordering_template = """Task: Repeat all provided sentences in the right order based on the provided rules. There is variable [VAR:animal].
Rules: If [VAR:animal] is cat then the text begins with the sentence about temperature.
[VAR:animal] is {A}
Sentences: {selection_A}
Answer:
"""
ordering_selection = PromptTemplate.from_template(ordering_template)
ordering_chain = LLMChain(llm=llm, prompt=ordering_selection, output_key="ordering")
# If [VAR:animal] is bird then the sentece about temperature is the last one. last or end or something not really working

#  is first one then the others

ordering_template2 = """"Taks: If the text contains a duplicite sentence, remove the second one.
Text: {ordering}
Answer:
"""
ordering_selection2 = PromptTemplate.from_template(ordering_template2)
ordering_chain2 = LLMChain(llm=llm, prompt=ordering_selection2, output_key="ordering2")



In [ ]:
def evaluate_precip(chain, data, phenomenon):
    correct = 0
    answers = []
    
    for i, row in data.iterrows():
        a = chain.run({"T":row.Temperature, "P":row.Precipitation, "W":row.Wind, "S":row.Sky, "L":row.Place, "A":row.Animal})
        answers.append(a)

        # why did I did this? I can just use dataframe...
        if phenomenon=="precipitation":
            if row.Precipitation == 0 or row.Sky == 'clear':
                label = "no rain and no snow"
            elif row.Temperature < 0:
                label = "snow"
            elif row.Place == "mountains":
                label = "storm"
            else:
                label = "rain"
                
            if 'There is no rain and no snow' in a and label=="no rain and no snow":
                correct +=1
            elif ('There is snow' in a or a=='There will be snow.') and label=="snow":
                correct += 1
            elif ('There is rain' in a or a=='There will be rain.') and label=="rain":
                correct += 1
            elif 'There is storm' in a  and label=="storm":
                correct += 1
            elif "here is "+label in a:
                correct +=1
            elif 'no snow and no rain' and label == "no rain and no snow":
                correct +=1
            elif ("rain"==a or "Rain"==a ) and label =="rain":
                correct +=1
            elif ("snow"==a or "Snow"==a) and label=="snow":
                correct +=1
            elif "nothing"==a and label=="no rain and no snow":
                correct +=1
            else:
                print(row.Sky, row.Temperature, row.Precipitation, row.Place)
                print(a)
                print(label)

        elif phenomenon=="wind":
            if row.Wind < 15:
                label = "no wind"
            elif row.Wind > 50:
                label = "strong wind"
            else:
                label = "wind"
            
            if 'There is strong wind' in a and label=="strong wind":
                correct +=1
            elif 'There is no wind' in a and label=="no wind":
                correct += 1
            elif 'There is wind' in a and label=="wind":
                correct +=1
            else:
                print(row.Wind)
                print(a)
                print(label)

        elif phenomenon=="sky":
            label = row.Sky
            if row.Place == 'city' or row.Animal=='bird':
                label = 'smog'
            if row.Place=='beach' and row.Animal=='cat':
                label = 'mist'

            if 'There is mist' in a and label=="mist":
                correct +=1
            elif 'The sky is '+str(row.Sky) in a and label==row.Sky:
                correct += 1
            elif 'There is smog' in a and label=="smog":
                correct +=1
            else:
                print(row.Sky)
                print(a)
                print(label)

        elif phenomenon=="ordering":
            if row.Animal == "cat" and row.Sky != 'clear':
                label= 'T, S, P, W'
            elif row.Animal == "bird" and row.Sky != 'clear':
                label = 'W, S, P, T'
            elif row.Sky == "clear":
                label = 'P, S, T, W'
            else: label ='S, P, T, W'

            if label in a:
                correct+=1
            elif row.Animal =='cat' and row.Sky == 'clear':
                if label == 'T, S, P, W' or 'P, S, T, W':
                    correct +=1
            elif row.Animal =='bird' and row.Sky == 'clear':
                if label == 'W, S, P, T' or 'P, S, T, W':
                    correct +=1
            else: 
                print(row.Animal, row.Sky)
                print(a)

        # CSA will do in the dataframe
        
            
    
    return answers, correct/len(data)

In [ ]:
%%time

#primary_precipitation_A, primary_precipitation_P = evaluate_precip(P_selection_chain, dataset)
#print('primary:', primary_precipitation_P)
results, percentage = evaluate_precip(CSP_chain, dataset, "precipitation") 
print('percentage result:', percentage)

#dataset['CSP'] = results

dataset.head()

In [ ]:
S2 = []
for i, row in dataset.iterrows():
    a = ordering_chain2.run({'wind_select':row.CSW, 'T':row.Temperature, 'P':row.Precipitation, 'sky_select':row['CSS both'], 'ordering':row.S1, 'precipitation2':row.CSP, 'W':row.Wind})
    S2.append(a)
    
dataset['S2'] = S2
dataset.head()

In [ ]:
ok_data = dataset[~(dataset.S2.str.contains('W') | dataset.S2.str.contains('T,') | dataset.S2.str.contains('S') | dataset.S2.str.contains('P'))] # 58x weird letter
ok_data = ok_data[((ok_data.S2.str.contains('rain')) | (ok_data.S2.str.contains('snow')))]
ok_data = ok_data[(ok_data.S2.str.contains('temperature'))]
ok_data = ok_data[(ok_data.S2.str.contains('sky is')) | (ok_data.S2.str.contains('mist')) | (ok_data.S2.str.contains('smog'))]
ok_data = ok_data[(ok_data.S2.str.contains('wind'))] #199
len(ok_data)/len(dataset)